In [4]:
from matplotlib.colors import rgb2hex
from sklearn.cluster import KMeans
import numpy as np
import re

In [31]:
def get_dominant_color(css_gradients):
    # Extracting colors from the CSS gradients
    colors = []
    for gradient in css_gradients:
        matches = re.findall(r'rgba\((\d+),\s*(\d+),\s*(\d+),\s*[\d.]+\)', gradient)
        colors.extend([tuple(map(int, match)) for match in matches])

    # Clustering to find dominant color
    kmeans = KMeans(n_clusters=1)
    kmeans.fit(colors)
    dominant_color = kmeans.cluster_centers_[0].astype(int)

    return tuple(dominant_color)

def rgb_to_hex(rgb):
    """Convert RGB to Hex color."""
    return rgb2hex([x/255.0 for x in rgb])

def approximate_color_name(rgb):
    """Approximate color name based on the RGB values."""
    # Color names and their RGB values
    color_names = {
        "01-Light": (255, 255, 255),
        "02-Lime": (123, 237, 159),
        "03-Green": (46, 213, 115),
        "04-Golden": (236, 204, 104),
        "05-Coral": (255, 127, 80),
        "06-Orange": (255, 165, 2),
        "07-Tomato": (255, 99, 72),
        "08-Watermelon": (255, 71, 87),
        "09-Blue": (112, 161, 255),
        "10-Sky": (55, 66, 250),
        "11-Grisaille": (87, 96, 111),
        "12-Prestige": (47, 53, 66),
    }

    # Find the closest color
    min_distance = float('inf')
    closest_color = None
    for name, color in color_names.items():
        distance = sum((a-b)**2 for a, b in zip(rgb, color))
        if distance < min_distance:
            min_distance = distance
            closest_color = name

    return closest_color

In [32]:
import json

css_gradients = []
# Open the file and load the data
with open('./gradients.json', 'r') as f:
    css_gradients = json.load(f)

In [ ]:
# Extracting dominant colors for each gradient
dominant_colors = [get_dominant_color([gradient['background']]) for gradient in css_gradients]

# Preparing the response with color codes and names
for i, color in enumerate(dominant_colors):
    hex_color = rgb_to_hex(color)
    color_name = approximate_color_name(color)
    css_gradients[i]['dominant'] = hex_color
    css_gradients[i]['color'] = color_name

# Write the updated data back to the file
with open('./gradients.json', 'w') as f:
    json.dump(css_gradients, f, indent=4)

In [34]:
def remove_similar_gradients(gradients):
    seen_colors = set()
    seen_names = set()
    unique_gradients = []

    for gradient in gradients:
        color = gradient['dominant']
        name = gradient['name']

        if color not in seen_colors and name not in seen_names:
            unique_gradients.append(gradient)
            seen_colors.add(color)
            seen_names.add(name)

    return unique_gradients

# Open the file and load the data
with open('./gradients.json', 'r') as f:
    data = json.load(f)

# Remove similar gradients
data = remove_similar_gradients(data)

# Write the updated data back to the file
with open('./gradients.json', 'w') as f:
    json.dump(data, f, indent=4)

In [35]:
from matplotlib.colors import rgb_to_hsv
from itertools import groupby

# Open the file and load the data
with open('./gradients.json', 'r') as f:
    data = json.load(f)

# Convert hex color to RGB
def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

# Sort the data by color
data = sorted(data, key=lambda x: x['color'])

# Write the sorted data back to the file
with open('./gradients.json', 'w') as f:
    json.dump(data, f, indent=4)